In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, roc_auc_score

%matplotlib inline

In [2]:
df = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [3]:
# See if there are any NaNs
df.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [4]:
def create_dummy(df, cat_cols, dummy_na):
    '''
    INPUT:
    df - pandas dataframe with categorical variables to dummy
    cat_cols - list of strings associated with the names of the categorical columns
    dummy_na - Bool holding whether you want to dummy NA vals of categorical columns or not
    
    OUTPUT:
    df - a new dataframe that has the following characteristics:
            1. contains all columns that were not specified as categorical
            2. dummy columns for each of the categorical columns in cat_cols
            3. if dummy_na is True - it also contains dummy columns for the NaN values
            4. Use a prefix of the column name with an underscore (_) for separating 
    '''
    
    df = pd.concat([df.select_dtypes(exclude='object'), pd.get_dummies(df[cat_cols], prefix_sep='_', drop_first=False, dummy_na=dummy_na)], axis=1)
    return df

# Data preprocessing

In [5]:
# 1. Remove rows with NaNs in the Response column (here there aren't any NaNs)
df = df.dropna(subset=['Response'], axis=0)

In [6]:
# 2.  

# Create X with only feature columns
X = df.drop(['Response','id'], axis=1)
# Create y with Response column
y = df['Response'] 
# Drop 'id' column also from df_test
df_test = df_test.drop(['id'], axis=1)

### Categorical values - method 1:
#### (This is not the chosen preprocessing method in the analysis)

One way of dealing with categorical values would be to create dummy columns only for the variables with initial dtype=object (here 'Gender', 'Vehicle_Age', 'Vehicle_Damage'). Then we can fill any NaNs in the remaining numerical and categorical variables using mode() (the mean wouldn't make sense in the case of the categorical variables).

In [7]:
# 3-1. Select categorical variables 

#cat_vars1 = X.select_dtypes(include=['object']).columns

In [8]:
# 4-1. Create dummy columns

#X = create_dummy(X, cat_vars1, dummy_na=True)
#df_test = create_dummy(df_test, cat_vars1, dummy_na=True) 

In [9]:
# 5-1. Fill any numerical/categorical NaNs with mode()

#fill_mean = lambda col: col.fillna(col.mode())
#X = X.apply(fill_mean, axis=0)
#df_test = df_test.apply(fill_mean, axis=0)

### Categorical values - method 2:
#### (This is the chosen preprocessing method in the analysis)

Another way of dealing with all categorical values would be to convert all of them to dtype=object and create dummy columns for all categorical variables. Then we can fill any NaNs in the remaining numerical variables using mean().



In [10]:
# 3-2. Convert all categorical variables to dtype=object 

X = X.astype({'Driving_License': object, 'Region_Code': object, 'Previously_Insured': object, 'Policy_Sales_Channel': object})
df_test = df_test.astype({'Driving_License': object, 'Region_Code': object, 'Previously_Insured': object, 'Policy_Sales_Channel': object})

cat_vars2 = X.select_dtypes(include=['object']).columns

In [11]:
# 4-2. Create dummy columns

X = create_dummy(X, cat_vars2, dummy_na=True)
df_test = create_dummy(df_test, cat_vars2, dummy_na=True) 

# Check if there are differences in the dummy column names of the two dataframes.
# If a dataframe misses a dummy column, then add it and fill it with zeros.
for col in set(X.columns).symmetric_difference(set(df_test.columns)): 
    if col in X.columns:
        df_test[col]=0
    else:
        X[col]=0

In [12]:
# 5-2. Fill any numerical NaNs with mean()

fill_mean = lambda col: col.fillna(col.mean())
X = X.apply(fill_mean, axis=0)
df_test = df_test.apply(fill_mean, axis=0)

In [13]:
# 6. Split into train/validation/test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.20, random_state=42)
X_test = df_test.copy()

In [14]:
# 7. Instantiate model
lm_model = LinearRegression(normalize=True)

In [15]:
# 8. Fit to model
lm_model.fit(X_train, y_train)

LinearRegression(normalize=True)

# Results

In [16]:
# 9. Predict using validation set
y_val_preds = lm_model.predict(X_val)

In [17]:
# 10. Score model predictions
score = roc_auc_score(y_val, y_val_preds) # Suggested Kaggle metric

In [18]:
# 11. Predict on unknown test data
y_test_preds = lm_model.predict(X_test)

In [19]:
score

0.848730419196233